In [43]:
import numpy as np
from random import *
"""
NN.py
====================================
Classes and methods for a neural network
"""
class NeuralNetwork:
    """Class that creates a NN and includes methods to train and test"""
    def __init__(self, setup=[[68,25,"sigmoid",0],[25,1,"sigmoid",0]],lr=.05,seed=1,error_rate=0,bias=1,iters=500,lamb=.00001,simple=0):
        #Note - these paramaters are examples, not the required init function parameters
        self._lr = lr
        self._seed = seed
        self._error_rate = error_rate
        self._bias = bias
        self._iters = iters
        self._lamb = lamb
        self._simple = simple
        
        

        # network is represented as a list of layers,
        # where layers are a list of nodes, where nodes
        # are a list of weights.
        # network = [ [[w1,w2...], [w1,w2...]] <- layer1
        #             [[w1,w2...], [w1,w2...]] <- layer2
        #           ]
        network = []
        
        # initialize the given number of layers with weights
        for layer in setup:
            network.append(self.make_weights(n_inputs=layer[0],n_nodes=layer[1]))
        
        self._network = network
        
    @property
    def network(self):
        return self._network

    @network.setter
    def network(self, network):
        self._network = network

    @property
    def lr(self):
        return self._lr

    @lr.setter
    def lr(self, lr):
        self._lr = lr

    @property
    def bias(self):
        return self._bias

    @bias.setter
    def bias(self, bias):
        self._bias = bias 
    
    @property
    def seed(self):
        return self._seed

    @seed.setter
    def bias(self, seed):
        self._seed = seed 
        
    def make_weights(self,n_inputs, n_nodes):
        """
        Generates random weights for the network initialization

        Parameters
        ---------
        n_inputs
            Number of input nodes to this layer
        n_nodes
            Number of nodes to generate weights for
            
        Returns
        ---------
        Layer with random weights initialized for each node
        """
        seed(self.seed)
        layer = []
        
        # Get n_inputs random numbers between 0 and 1 for each node
        for i in range(n_nodes):
            layer.append([random() for i in range(n_inputs)])
        
        return layer

    def feedforward(self, data):
        """
        Takes in data and passes it through the NN

        Parameters
        ---------
        data
            One datapoint
            
        Returns
        ---------
        The output(s) of the final layer in the network
        """
        inputs = data
        
        # pass data through all layers
        for layer in self.network:
            next_inputs = []
            for node_weights in layer:
                sum = self.bias
                for i in range(len(inputs)): # multiply inputs by weights and add to sum
                    sum += inputs[i]*node_weights[i]
                next_inputs.append(sum)
            inputs = next_inputs
        
        # input should now be the final layer output
        return inputs
    
    def backprop(self):
        pass

    def fit(self):
        pass

    def predict(self):
        pass

def activation(input, weights):
    """
    Applies the weights and biases to inputs of a node

    Parameters
    ---------
    input
        The input values
    weights
        a list of weights to be applied
    
    Returns
    ---------
    sum of the weighted inputs plus bias
    """
    sum = self.bias
    
    # multiply inputs by weights and add to sum
    for i in range(len(input)):
        sum += input[i]*weights[i]

    return sum

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x)*(1 - sigmoid(x))

In [45]:
#from scripts import NN

def test_make_weights():
    nn = NeuralNetwork(setup=[[8,3,"sigmoid",0],[3,8,"sigmoid",0]])
    assert len(nn.network) == 2
    assert len(nn.network[0]) == 3
    assert len(nn.network[1]) == 8
    assert len(nn.network[0][0]) == 8
    assert len(nn.network[1][0]) == 3

def test_feedforward():
    nn = NeuralNetwork()
    # a 2x1x2 network
    nn.network = [ [[0.5, 0.5]], # first hidden layer (1 node)
                  [[0.5], [0.5]] ] # output later (2 nodes)
    assert nn.feedforward([1,1]) == [2.0, 2.0]
    
def test_encoder():
    assert True

def test_encoder_relu():
    assert True

def test_one_d_ouput():
    assert True


In [44]:
nn = NeuralNetwork()
# a 2x1x2 network
nn.network = [ [[0.5, 0.5]], [[0.5], [0.5]] ]
nn.feedforward([1,1])

[2.0, 2.0]

In [46]:
test_feedforward()
